In [1]:
import pandas as pd

In [3]:
df1 = pd.DataFrame({'a': [1,2,3,4,5,6], 'b': ['a', 'b', 'c', 'd', 'e', 'f'], 'c': ['p'+str(x) for x in range(6)], 'd': ['ah' for x in range(6)]})

In [4]:
df2 = pd.DataFrame({'g': [1,2,3,5,6], 'h': ['hi', 'wow', 'potato', 'calendar', 'moment']})

In [28]:
def jaccard(s1, s2):
    return len(s1 & s2) / len(s1 | s2)

def automerge(d1, d2):
    # for each column in d1, check if there are any columns in
    options = []
    d1pks = []
    d2pks = []
    for c1 in d1.columns:
        if len(d1[c1].unique()) == len(d1):
            d1pks.append(c1)
    for c2 in d2.columns:
        if len(d2[c2].unique()) == len(d2):
            d2pks.append(c2)
    # for each pk, check other table for possible joins
    for d1pk in d1pks:
        for c2 in d2.columns:
            if set(d2[c2]).issubset(set(d1[d1pk])):
                options.append({'left_on': d1pk, 'right_on': c2, 'how': 'right'})
            if jaccard(set(d2[c2]), set(d1[d1pk])) > 0.5:
                options.append({'left_on': d1pk, 'right_on': c2, 'how': 'outer'})
    for d2pk in d2pks:
        for c1 in d1.columns:
            if set(d1[c1]).issubset(set(d2[d2pk])):
                options.append({'left_on': c1, 'right_on': d2pk, 'how': 'left'})
            if jaccard(set(d1[c1]), set(d2[d2pk])) > 0.5:
                options.append({'left_on': c1, 'right_on': d2pk, 'how': 'outer'})
    output = []
    for option in options:
        output.append((option, pd.merge(df1, df2, **option)))
    return output

In [29]:
print("df1")
print(df1)
print("df2")
print(df2)

for method, df in automerge(df1, df2):
    print(method)
    print(df)

df1
   a  b   c   d
0  1  a  p0  ah
1  2  b  p1  ah
2  3  c  p2  ah
3  4  d  p3  ah
4  5  e  p4  ah
5  6  f  p5  ah
df2
   g         h
0  1        hi
1  2       wow
2  3    potato
3  5  calendar
4  6    moment
{'left_on': 'a', 'right_on': 'g', 'how': 'right'}
   a  b   c   d  g         h
0  1  a  p0  ah  1        hi
1  2  b  p1  ah  2       wow
2  3  c  p2  ah  3    potato
3  5  e  p4  ah  5  calendar
4  6  f  p5  ah  6    moment
{'left_on': 'a', 'right_on': 'g', 'how': 'outer'}
   a  b   c   d    g         h
0  1  a  p0  ah  1.0        hi
1  2  b  p1  ah  2.0       wow
2  3  c  p2  ah  3.0    potato
3  4  d  p3  ah  NaN       NaN
4  5  e  p4  ah  5.0  calendar
5  6  f  p5  ah  6.0    moment
{'left_on': 'a', 'right_on': 'g', 'how': 'outer'}
   a  b   c   d    g         h
0  1  a  p0  ah  1.0        hi
1  2  b  p1  ah  2.0       wow
2  3  c  p2  ah  3.0    potato
3  4  d  p3  ah  NaN       NaN
4  5  e  p4  ah  5.0  calendar
5  6  f  p5  ah  6.0    moment
